In [1]:
import random
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

#Практика 11.10

In [3]:
# Номер 1
# Функция для генерации выборок и анализа квантилей
def analyze_samples(distribution_func, dist_params, sample_size, alpha):
    # Генерация выборки
    sample = distribution_func(*dist_params, size=sample_size)

    # Определение квантилей
    q_1 = np.percentile(sample, 1)
    q_95 = np.percentile(sample, 95)

    # Определение доверительных интервалов
    mean = np.mean(sample)
    std_err = np.std(sample, ddof=1) / np.sqrt(sample_size)
    critical_value = stats.norm.ppf(1 - (1 - alpha) / 2)

    ci_lower = mean - critical_value * std_err
    ci_upper = mean + critical_value * std_err

    return q_1, q_95, ci_lower, ci_upper, sample

# Определение размера выборки для доверительных интервалов
def calculate_sample_size(alpha, margin_of_error, std_dev):
    z_score = stats.norm.ppf(1 - (1 - alpha) / 2)
    sample_size = (z_score * std_dev / margin_of_error) ** 2
    return int(np.ceil(sample_size))

# Параметры
alpha_values = [0.90, 0.95, 0.99]  # Доверительные вероятности
delta_values = [0.1, 0.05, 0.01]    # Погрешности
sample_sizes = []
distributions = {
    'uniform': (np.random.uniform, (0, 1)),
    'normal': (np.random.normal, (0, 1)),
    'cauchy': (np.random.standard_cauchy, ())
}

# Анализ
results = {}

for dist_name, (distribution_func, dist_params) in distributions.items():
    results[dist_name] = []

    # Размеры выборок для каждого уровня доверия
    for delta in delta_values:
        sample_size = calculate_sample_size(alpha=0.95, margin_of_error=delta, std_dev=1)  # Предполагаем, что std_dev = 1 для упрощения
        q_1, q_95, ci_lower, ci_upper, sample = analyze_samples(distribution_func, dist_params, sample_size, alpha=0.95)

        results[dist_name].append({
            'sample_size': sample_size,
            'q_1': q_1,
            'q_95': q_95,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'sample': sample,
            'within_bounds': ci_lower >= sample.min() and ci_upper <= sample.max()
        })

# Вывод результатов
for dist_name, dist_results in results.items():
    print(f"\nDistribution: {dist_name}")
    for result in dist_results:
        print(f"Sample Size: {result['sample_size']}, "
              f"1% Quantile: {result['q_1']}, "
              f"95% Quantile: {result['q_95']}, "
              f"CI: ({result['ci_lower']}, {result['ci_upper']}), "
              f"Within Bounds: {result['within_bounds']}")


Distribution: uniform
Sample Size: 385, 1% Quantile: 0.014812488498896293, 95% Quantile: 0.9422811700021708, CI: (0.4647379463994217, 0.5218523576179057), Within Bounds: True
Sample Size: 1537, 1% Quantile: 0.0047236527462092455, 95% Quantile: 0.9587930075785882, CI: (0.4845894244750031, 0.513494867805992), Within Bounds: True
Sample Size: 38415, 1% Quantile: 0.009230222715573565, 95% Quantile: 0.9491966333377452, CI: (0.49676326766168577, 0.5025202844779927), Within Bounds: True

Distribution: normal
Sample Size: 385, 1% Quantile: -2.1997360002291186, 95% Quantile: 1.7095260567080321, CI: (-0.1083484392218544, 0.09501727051542798), Within Bounds: True
Sample Size: 1537, 1% Quantile: -2.1858899293080976, 95% Quantile: 1.696137186941254, CI: (-0.03619898720694227, 0.06373015470597006), Within Bounds: True
Sample Size: 38415, 1% Quantile: -2.341855901313372, 95% Quantile: 1.6600422137101103, CI: (-0.0039212865923663656, 0.016107336132470097), Within Bounds: True

Distribution: cauchy
Sa

In [4]:
# Номер 2
# Функция для генерации выборок и вычисления статистик
def generate_samples(distribution_func, params, size):
    return distribution_func(*params, size=size)

# Метод Монте-Карло
def monte_carlo(samples, n_iterations=10000):
    estimates = []
    for _ in range(n_iterations):
        sample = np.random.choice(samples, size=len(samples), replace=True)
        estimates.append(np.mean(sample))
    return np.std(estimates)

# Метод линеаризации
def linearization_method(samples):
    mean = np.mean(samples)
    std_dev = np.std(samples, ddof=1)

    # Погрешность для математического ожидания
    std_err_mean = std_dev / np.sqrt(len(samples))

    # Погрешность для дисперсии
    var_estimate = np.var(samples, ddof=1)
    std_err_variance = (2 * var_estimate**2 / (len(samples) - 1))**0.5

    return std_err_mean, std_err_variance

# Метод Крейновича
def kreinovich_method(samples):
    n = len(samples)
    mean = np.mean(samples)
    std_dev = np.std(samples, ddof=1)

    # Погрешность для математического ожидания
    delta_mean = std_dev / np.sqrt(n)

    # Погрешность для дисперсии
    var_estimate = np.var(samples, ddof=1)
    delta_variance = (2 * var_estimate**2) / (n - 1)

    return delta_mean, delta_variance

# Параметры для генерации выборки
distribution_func = np.random.normal
params = (0, 1)  # Среднее и стандартное отклонение
sample_size = 100  # Размер выборки

# Генерация выборки
samples = generate_samples(distribution_func, params, sample_size)

# Оценка погрешностей
monte_carlo_error = monte_carlo(samples)
linearization_errors = linearization_method(samples)
kreinovich_errors = kreinovich_method(samples)

# Вывод результатов
print(f"Метод Монте-Карло: Погрешность = {monte_carlo_error}")
print(f"Метод линеаризации: Погрешность математического ожидания = {linearization_errors[0]}, "
      f"Погрешность дисперсии = {linearization_errors[1]}")
print(f"Метод Крейновича: Погрешность математического ожидания = {kreinovich_errors[0]}, "
      f"Погрешность дисперсии = {kreinovich_errors[1]}")

Метод Монте-Карло: Погрешность = 0.10645595751599854
Метод линеаризации: Погрешность математического ожидания = 0.10588412270879073, Погрешность дисперсии = 0.1593525750651481
Метод Крейновича: Погрешность математического ожидания = 0.10588412270879073, Погрешность дисперсии = 0.025393243179893665
